<center><img src="../../fig/ICHEC_Logo.jpg" alt="Drawing" style="width: 500px;"/>

<center><img src="../../fig/notebooks/MPI_Logo.png" alt="Drawing" style="width: 250px;"/>

# <center>Exercises

Use the template submission script below to submit your files to the specialised queue. Substitute the `{# MY_JOB #}`, `{# MY_FILE #}`, `{# MY_ENV #}`, `{# MY_COMMAND #}`, strings for appropriate job, file name, as well as correct commands and environment. 

```slurm
#!/bin/bash
#SBATCH --nodes=1
#SBATCH --time=00:10:00
#SBATCH -A course
#SBATCH --job-name={# MY_JOB #}
#SBATCH -p CourseDevQ
#SBATCH --reservation=CourseMay


module purge
module load conda
module list

source activate {# MY_ENV #}

cd $SLURM_SUBMIT_DIR

{# MY_COMMAND #} {# MY_FILE #}.py

exit 0
```
    




***
# <center> <b>MPI (Message Passing Interface)</b>
    
### <center><b>Exercise 1: Multiplying an array</b>

#### <center> 5 minutes

Using the code just covered in the lecture, found below, create a list from 1-4, and use a for loop to multiply the element of the list by the rank. Add a print statement to show how the array changes depending on the rank.

```python
from mpi4py import MPI

# communicator containing all processes
comm = MPI.COMM_WORLD 

size = comm.Get_size()
rank = comm.Get_rank()

# TODO: Create array and loop

print("I am rank %d in group of %d processes" % (rank, size))
# TODO: Add print statement
```

***

### <center><b>Exercise 2: Pairwise Exchange</b>

#### <center> 10 minutes
    


Copy the code block below into a file caled `pairwise_exchange.py`, implement a one-dimensional pairwise exchange of the ranks of n processes with periodic boundary conditions. That is, each process communicates its rank to each of its topological neighbours.

```python
from mpi4py import MPI

comm = MPI.COMM_WORLD 
rank = comm.Get_rank()
size = comm.Get_size()

# TODO: Define neighbours

# TODO: Use send and recv to 

print ('I am process ', rank, ', my neighbours are processes', left, ' and ', right)
```

For 4 processes, the output should be similar to;
```
I am process  1 , my neighbours are processes 0  and  2
I am process  3 , my neighbours are processes 2  and  0
I am process  0 , my neighbours are processes 3  and  1
I am process  2 , my neighbours are processes 1  and  3
```

***
### <center><b>Exercise 3: Parallel Sum</b>

#### <center> 25 minutes
    


Implement the parallel sum case study. That is;
- take the code block below
- implement the appropriate send/receive to distribute the data from rank 0
- perform the partial sum on each process
- implement the appropriate send/receive to gather the data at rank 0
- compute the final sum from the gathered data on rank 0

(Bonus) For added difficulty (and best practice), try to implement the same approach with an arbitrary number of processes. 

```python
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

local_data = np.empty( , dtype=int)

# Distribute data
if rank==0:
    data = np.arange(100, dtype=int)
    local_data = data[] # TODO: choose appropriate range
    # TODO: Communicate
else:    
    # TODO: Communicate

# TODO: Compute local and partial sums
local_sum = np.sum(local_data)

# Collect data
if rank==0:
    partial_sums = np.empty( , dtype=int)
    # TODO: Communicate
    
    # TODO: Calculate total sum
    
    # TODO: Print result
else:
    # TODO: Communicate
```

***
### <center><b>Exercise 4: SendRecv pairwise exchange</b>


#### <center> 10 minutes
    

Modify the pairwise exchange code from Exercise 2 to use the combined send/recv communication.


***
### <center><b>Exercise 5: Non-Blocking Communication in a Ring</b>


#### <center> 10 minutes
    
    
- A set of processes are arranged in a ring
- 1. Each process stores its rank in MPI.COMM_WORLD into an integer variable, `snd_buf`
- 2. Each process passes this `snd_buf` to the neighbour on its right
- 3. Prepares for the next iteration
- 4. Each processor calculates the sum of all values
- Steps 2-4 are repeated with n number of times (number of processes)
- Each process calculates the sum of all ranks

The code below can work as we are only sending a small message. But it is still wrong.
    
Using a *synchronous send*, `Ssend`, will definitely cause a deadlock. Using a regular `Send` will at some point cause a deadlock, but not 100% of the time.
     
**NB: For this exercise only use `Issend`, which is used to demonstrate a deadlock is the non-blocking routine is incorrectly used. Real applications use `Isend`, not `Issend`**
    

```python
from mpi4py import MPI
import numpy as np

rcv_buf = np.empty((), dtype=np.intc)
#rcv_buf = np.empty((), dtype=np.intc) # uninitialized 0 dimensional integer array
status = MPI.Status()

comm = MPI.COMM_WORLD
my_rank = comm.Get_rank()
size = comm.Get_size()

right = (my_rank+1) % size
left  = (my_rank-1+size) % size

sum = 0
snd_buf = np.array(my_rank, dtype=np.intc) # 0 dimensional integer array with 1 element initialized with the value of my_rank

for i in range(size):
   comm.Send((snd_buf, 1, MPI.INT), dest=right, tag=17)
   
   #request = comm.Isend((snd_buf, 1, MPI.INT), dest=right, tag=17)


   comm.Recv((rcv_buf, 1, MPI.INT), source=left,  tag=17, status=status)
   #request.Wait(status)
   np.copyto(snd_buf, rcv_buf) # We make a copy here. What happens if we write snd_buf = rcv_buf instead?
   sum += rcv_buf
print(f"PE{my_rank}:\tSum = {sum}")
```

***
### <center><b>Exercise 6a: P2P to Gather


#### <center> 15 minutes
    

The Send and Receive code here works perfectly as is, but using `gather` would certainly make things much cleaner, and we only have to call one MPI command.
    
Substitute the point-to-point communication with one call to `gather`

```python
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
my_rank = comm.Get_rank()
num_procs = comm.Get_size()

# Each process assigned some work
res = np.array(my_rank**2, dtype=np.double)
print(f"I am process {my_rank} out of {num_procs}, result={res:f}")
if (my_rank == 0):
   res_arr = np.empty(num_procs, dtype=np.double)

# TODO: Substitute the following block with MPI_Gather
###
if (my_rank != 0):  
   # Sending some results from all processes (except 0) to process 0:
   comm.Send((res, 1, MPI.DOUBLE), dest=0, tag=99)
else: 
   res_arr[0] = res # process 0's own result
   # Receive all the messages
   for rank in range(1,num_procs):
      # Result of processes 1 -> n
      comm.Recv((res_arr[rank:], 1, MPI.DOUBLE), source=rank, tag=99, status=None) 
###

if (my_rank == 0):
   for rank in range(num_procs):
      print(f"I'm proc 0: result of process {rank} is {res_arr[rank]:f}")
```

### <center><b>Exercise 6b (Optional): Parallel Sum with Collective Communication 

Modify the parallel sum code from the second exercise to use replace the point-to-point communication with collective communication. Preferably, this should abstracted to an arbitrary number of processes. Try with different methods, which one(s) produce the expected results?
    

***
### <center><b>Exercise 7: Global reduction 
    
#### <center> 10 minutes </center>
    
The communication around a ring covered in Exercise 5 can be further simplified using a global reduction. modify your solution to Exercise 5 by:
    
- Determine the quantity of code that needs to be replaced by the global reduction
- Using `Allreduce` to call the collective reduction routine


***
### <center><b>Exercise 8: Communicators
    
#### <center> 20 minutes </center>
    
Modify the Allreduce program. You will need to split the communicator MPI_COMM_WORLD into 1/3 and 2/3, so the color variable for the input for `Split` is;
    
color = (rank > [$\frac{size -1}{3}$])
    
- Calculate **sumA** and **sumB** over all processes within each sub-communicator
- Run with 12 processes to produce the following results:
    - **sumA:** Sums up ranks in MPI_COMM_WORLD: 4 & 8 split with world ranks;
        - 0 -> 3  = 6
        - 4 -> 11 = 60 
    - **sumB:** Sums up ranks within new sub-communicators: 4 & 8 split with sub-comm ranks:
        - 0 -> 3  = 6
        - 0 -> 7  = 28

```python
sumA = np.empty((), dtype=np.intc)
sumB = np.empty((), dtype=np.intc)

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = np.array(comm.Get_rank(), dtype=np.intc)

# TODO
# 1. Define 'color' variable
# 2. Define your sub-communicator using a split
# 3. Define your new size and ranks
# 4. Keep track of your variable names for clarity

# Compute sum of all ranks.
comm.Allreduce(rank, (sumA, 1, MPI.INT), MPI.SUM) 
comm.Allreduce(rank, (sumB, 1, MPI.INT), MPI.SUM)

print("PE world:{:3d}, color={:d} sub:{:3d}, SumA={:3d}, SumB={:3d} in comm_world".format( 
          rank, "TODO: color", "TODO: sub_rank", sumA, sumB))
```

***
### <center><b> Easier Exercise: Collective Communication </b></center>

- In this exercise you will use different routines for collective communication. Use the skeleton code to get started.
  
  
A) First, write a program where rank 0 sends an array containing integers from 0 to 7 to all other ranks using collective communication. Use four cores.
[Hint: Use broadcasting]

- From these arrays create the initial arrays shown below.

B)

|        |  |  |  |  |  |  |  |  |
|--------|--|--|--|--|--|--|--|--|
|Task 0: | 0| 1| 2| 3| 4| 5| 6| 7|
|Task 1: | 8| 9|10|11|12|13|14|15|
|Task 2: |16|17|18|19|20|21|22|23|
|Task 3: |24|25|26|27|28|29|30|31|


- Each task should receive a buffer for eight elements with each one initialised to -1. 

- Implement a program that sends and receives values from the data arrays to receive buffers using single collective routines so that the receive buffers will have the following values;

C)

|        |  |  |  |  |  |  |  |  |
|--------|--|--|--|--|--|--|--|--|
|Task 0: | 0| 1|-1|-1|-1|-1|-1|-1|
|Task 1: | 2| 3|-1|-1|-1|-1|-1|-1|
|Task 2: | 4| 5|-1|-1|-1|-1|-1|-1|
|Task 3: | 6| 7|-1|-1|-1|-1|-1|-1|

- [Hint: Use `comm.Scatter()`]

D)

|        |  |  |  |  |  |  |  |  |
|--------|--|--|--|--|--|--|--|--|
|Task 0: |-1|-1|-1|-1|-1|-1|-1|-1|
|Task 1: | 0| 1| 8| 9|16|17|24|25|
|Task 2: |-1|-1|-1|-1|-1|-1|-1|-1|
|Task 3: |-1|-1|-1|-1|-1|-1|-1|-1|

- [Hint: Use `comm.Gather()`]

E)

|        |  |  |  |  |  |  |  |  |
|--------|--|--|--|--|--|--|--|--|
|Task 0: | 8|10|12|14|16|18|20|22|
|Task 1: |-1|-1|-1|-1|-1|-1|-1|-1|
|Task 2: |40|42|44|46|48|50|52|54|
|Task 3: |-1|-1|-1|-1|-1|-1|-1|-1|

- [Hint: Create two communicators and use `comm.Reduce()`]

```python
from mpi4py import MPI
import numpy
from sys import stdout

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

assert size == 4, 'Number of MPI tasks has to be 4.'

if rank == 0:
    print('A)Broadcast:')

# TODO: create data vector at task 0 and send it to everyone else
#       using collective communication
if rank == 0:
    data = ...
else:
    data = ...
...
print('  Task {0}: {1}'.format(rank, data))


# Prepare data vectors ..
data = ...  # TODO: create the data vectors
# .. and receive buffers
buff = numpy.full(8, -1, int)

# ... wait for every rank to finish ...
comm.barrier()
if rank == 0:
    print('')
    print('-' * 32)
    print('')
    print('B) Initial Data vectors:')
print('  Task {0}: {1}'.format(rank, data))
comm.barrier()
if rank == 0:
    print('')
    print('-' * 32)
    print('')
    print('C) Scatter:')

# TODO: how to get the desired receive buffer using a single collective
#       communication routine?
...
print('  Task {0}: {1}'.format(rank, buff))

# ... wait for every rank to finish ...
buff[:] = -1
comm.barrier()
if rank == 0:
    print('')
    print('-' * 32)
    print('')
    print('D) Gather:')

# TODO: how to get the desired receive buffer using a single collective
#       communication routine?

...
print('  Task {0}: {1}'.format(rank, buff))

# ... wait for every rank to finish ...
buff[:] = -1
comm.barrier()
if rank == 0:
    print('')
    print('e)')

# TODO: how to get the desired receive buffer using a single collective
#       communication routine?
...
print('  Task {0}: {1}'.format(rank, buff))

```

***
***

## <center><b>Heat Equation<b/>
    
The series of exercises covered here applies the methods covered during the lectures to the heat equation we have seena few time before. These can be completed in your own time or if you finish an exercise early, give these a go and see how you fare. The codes to modify everything can be found in [heat_equation.ipynb](./heat_equation/heat_equation.ipynb). The solutions for these are available in [soln](./soln).

***
### <center><b>Advanced Exercise 1: Point to Point</b>

<center><img src="../img/4.1.1.png" alt="Drawing" style="width: 350px;"/> </center>

- **<span style="color:green">halo region</span>**
- **<span style="color:purple">border region</span>**
- **<span style="color:blue">local region</span>**

Modify the [heat equation codebase](./heat_equation/heat_equation.ipynb) below to perform one-dimensional domain decomposition using MPI. This involves;
- splitting the grid between each process
- maintain halo regions as indicated in the above image for storing neighbouring border data
- perform an exchange of border data before each iteration of the grid (i.e, border data is sent to the halo region of the appropriate neighbour)
- transfer the final grid back to the root rank 

To do this, you will need to;
1. implement the `halo_exchange` function in `modules/comms.py`.
2. implement the `distribute_subfields` and `gather_subfields` functions in `modules/comms.py`. These should distribute the field from the rank 0 process to the rest and from the rest to the rank 0 process respectively.
2. modify `main.py` `main` function to only initialise the field on rank 0. Call the `distribute_subfields` and `gather_subfields` functions where appropriate. Only print and write to files from the rank 0 process.
4. modify the `iterate` function in `main.py` to call the `halo_exchange` function before each iteration.

***
### <center><b>Advanced Exercise 2: Non-Blocking Communication</b>

Further modify the [heat equation codebase](./heat_equation/heat_equation.ipynb) to use non-blocking communication for the halo exchange. Make sure not to access transferred data without waiting for it to arrive!

***
### <center><b>Advanced Exercise 3: Collective Communication</b>

Modify the grid distribution strategy used in the [heat equation codebase](./heat_equation/heat_equation.ipynb) to use collective communication. This will involve changes to the `distribute_subfields` and `gather_subfields` functions in `modules/comms.py`.